In [1]:
import numpy as np

In [2]:
#2 Загрузка данных
data = np.load('digitsMnist.npy', allow_pickle=True)

# Разделение данных
X_forTrain = data[0]
Y_forTrain = data[1]
X_forVal = data[2]
Y_forVal = data[3]

In [3]:
#3  Условная выборка цифр 3 и 8

train_mask = (Y_forTrain == 3) | (Y_forTrain == 8)
val_mask = (Y_forVal == 3) | (Y_forVal == 8)

X_forTrain = X_forTrain[train_mask]
Y_forTrain = Y_forTrain[train_mask]
X_forVal = X_forVal[val_mask]
Y_forVal = Y_forVal[val_mask]


In [4]:
#4 Переименование меток для бинарной классификации

Y_forTrain = np.where(Y_forTrain == 3, 0, 1)
Y_forVal = np.where(Y_forVal == 3, 0, 1)

In [5]:
#5 Нормализация данных

X_forTrain = X_forTrain / 255.0
X_forVal = X_forVal / 255.0


In [6]:
from sklearn.model_selection import train_test_split

#6 Разбиение на обучающую и тестовую выборки

X_train, X_test, Y_train, Y_test = train_test_split(
    X_forTrain, Y_forTrain, test_size=0.2, random_state=42
)


In [7]:
import torch

#7 Преобразование массивов в тензоры

X_train_tensors = torch.tensor(X_train, dtype=torch.float32)
X_test_tensors = torch.tensor(X_test, dtype=torch.float32)
Y_train_tensors = torch.tensor(Y_train, dtype=torch.long)
Y_test_tensors = torch.tensor(Y_test, dtype=torch.long)

X_val_tensors = torch.tensor(X_forVal, dtype=torch.float32)
Y_val_tensors = torch.tensor(Y_forVal, dtype=torch.long)


In [9]:
# 9-11 Создание TensorDataset для обучающей, тестовой и валидационной выборок

from torch.utils.data import TensorDataset

# Объединяем входные данные и метки в TensorDataset
trainDataset = TensorDataset(X_train_tensors, Y_train_tensors)
testDataset = TensorDataset(X_test_tensors, Y_test_tensors)
valDataset = TensorDataset(X_val_tensors, Y_val_tensors)


In [10]:
#12-14  Создание DataLoader для батчей
 
from torch.utils.data import DataLoader

# Размер батча (выберите оптимальный для вашей задачи, например, 64)
batch_size = 64

# Создаем DataLoader для обучающей, тестовой и валидационной выборок
train_loader = DataLoader(trainDataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(testDataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(valDataset, batch_size=batch_size, shuffle=False)


In [11]:
#15 Создание класса нейронной сети

import torch.nn as nn
import torch.nn.functional as F

class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)  # Входной слой: 784 нейрона, скрытый слой: 128 нейронов
        self.fc2 = nn.Linear(128, 64)       # Скрытый слой: 64 нейрона
        self.fc3 = nn.Linear(64, 2)         # Выходной слой: 2 класса (бинарная классификация)

    def forward(self, x):
        x = x.view(-1, 28 * 28)            # Преобразуем тензор в вектор
        x = F.relu(self.fc1(x))            # Функция активации ReLU для первого слоя
        x = F.relu(self.fc2(x))            # Функция активации ReLU для второго слоя
        x = self.fc3(x)                    # Выходной слой (без активации)
        return x


In [12]:
#16-19 Создание экземпляра сети, выбор оптимизатора и функции потерь

# Создание экземпляра сети
simplenet = SimpleNet()

# Выбор оптимизатора Adam с шагом обучения lr=0.001
optimizer = torch.optim.Adam(simplenet.parameters(), lr=0.001)

# Выбор функции потерь: перекрестная энтропия
criterion = nn.CrossEntropyLoss()


In [13]:
#20-21  Обучение модели

# Используем CPU для вычислений
device = torch.device('cpu')
simplenet.to(device)

# Количество эпох
num_epochs = 10

for epoch in range(num_epochs):
    simplenet.train()  # Включаем режим обучения
    train_loss = 0
    
    # Обучение на батчах
    for X_batch, Y_batch in train_loader:
        X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
        
        optimizer.zero_grad()               # Обнуляем градиенты
        outputs = simplenet(X_batch)        # Прогон вперед
        loss = criterion(outputs, Y_batch)  # Вычисляем функцию потерь
        loss.backward()                     # Вычисляем градиенты
        optimizer.step()                    # Обновляем параметры сети
        
        train_loss += loss.item() * X_batch.size(0)
    
    # Оценка на тестовой выборке
    simplenet.eval()  # Включаем режим оценки
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for X_batch, Y_batch in test_loader:
            X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
            outputs = simplenet(X_batch)
            loss = criterion(outputs, Y_batch)
            test_loss += loss.item() * X_batch.size(0)
            
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == Y_batch).sum().item()
    
    train_loss /= len(train_loader.dataset)
    test_loss /= len(test_loader.dataset)
    accuracy = correct / len(test_loader.dataset)

    print(f'Epoch [{epoch+1}/{num_epochs}], '
          f'Train Loss: {train_loss:.4f}, '
          f'Test Loss: {test_loss:.4f}, '
          f'Accuracy: {accuracy:.4f}')


Epoch [1/10], Train Loss: 0.1559, Test Loss: 0.1476, Accuracy: 0.9499
Epoch [2/10], Train Loss: 0.0782, Test Loss: 0.0692, Accuracy: 0.9762
Epoch [3/10], Train Loss: 0.0471, Test Loss: 0.0416, Accuracy: 0.9875
Epoch [4/10], Train Loss: 0.0274, Test Loss: 0.0326, Accuracy: 0.9900
Epoch [5/10], Train Loss: 0.0163, Test Loss: 0.0342, Accuracy: 0.9900
Epoch [6/10], Train Loss: 0.0125, Test Loss: 0.0305, Accuracy: 0.9917
Epoch [7/10], Train Loss: 0.0075, Test Loss: 0.0415, Accuracy: 0.9900
Epoch [8/10], Train Loss: 0.0055, Test Loss: 0.0433, Accuracy: 0.9912
Epoch [9/10], Train Loss: 0.0066, Test Loss: 0.0494, Accuracy: 0.9896
Epoch [10/10], Train Loss: 0.0060, Test Loss: 0.0349, Accuracy: 0.9921


In [14]:
#21 Оценка на валидационной выборке

val_loss = 0
correct = 0

with torch.no_grad():
    for X_batch, Y_batch in val_loader:
        X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
        outputs = simplenet(X_batch)
        loss = criterion(outputs, Y_batch)
        val_loss += loss.item() * X_batch.size(0)
        
        preds = torch.argmax(outputs, dim=1)
        correct += (preds == Y_batch).sum().item()

val_loss /= len(val_loader.dataset)
val_accuracy = correct / len(val_loader.dataset)

print(f'Validation Loss: {val_loss:.4f}, '
      f'Validation Accuracy: {val_accuracy:.4f}')


Validation Loss: 0.0324, Validation Accuracy: 0.9884


In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pytorch_lightning as pl
from torchmetrics import Accuracy
from pytorch_lightning.loggers import TensorBoardLogger

In [27]:
class SimpleNet(pl.LightningModule):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 2)
        self.criterion = nn.CrossEntropyLoss()
        self.accuracy = Accuracy(task='multiclass', num_classes=2)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        acc = self.accuracy(y_hat, y)
        self.log('val_loss', loss)
        self.log('val_acc', acc)

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=0.001)


In [28]:
logger = TensorBoardLogger("logs", name="simple_net")

trainer = pl.Trainer(
    max_epochs=10,
    logger=logger
)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [29]:
model = SimpleNet()
trainer.fit(model, train_loader, val_loader)


  | Name      | Type               | Params
-------------------------------------------------
0 | fc1       | Linear             | 100 K 
1 | fc2       | Linear             | 8.3 K 
2 | fc3       | Linear             | 130   
3 | criterion | CrossEntropyLoss   | 0     
4 | accuracy  | MulticlassAccuracy | 0     
-------------------------------------------------
108 K     Trainable params
0         Non-trainable params
108 K     Total params
0.435     Total estimated model params size (MB)


c:\Users\user\Desktop\study\python\venv\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 9: 100%|██████████| 181/181 [00:01<00:00, 152.17it/s, loss=0.000797, v_num=1]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 181/181 [00:01<00:00, 150.42it/s, loss=0.000797, v_num=1]


In [ ]:
!tensorboard --logdir logs/